In [1]:
import sys
sys.path.insert(0,'..')
from IPython.display import HTML, Image as Image1, display, clear_output
import ipywidgets as widgets
try:
    from urllib.request import urlretrieve
except ImportError:
    from urllib import urlretrieve
from PIL import Image
import os
import multiprocessing
from sklearn.model_selection import train_test_split  
from functools import partial
import numpy as np
from glob import glob
from sklearn import svm
import numpy.linalg
from PIL import Image
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from pathlib import Path

from dealing_with_images import square_resize_image
import image_embeddings
from shutil import copyfile

from matplotlib.pyplot import imread

In [12]:
# renaming images in dataset2
directories = next(os.walk('.'))[1]
directories.pop(-2)
for dirname in directories:
    for i, filename in enumerate(os.listdir(dirname)):
        os.rename(dirname + "/" + filename, dirname + "/" + str(i) + ".bmp")

normal_in_camera
nose_picking
eyes_touching
nobody
other_parts_face_touching
.ipynb_checkpoints
biting_nails


In [22]:
def has_transparency(img):
        if img.mode == "P":
            transparent = img.info.get("transparency", -1)
            for _, index in img.getcolors():
                if index == transparent:
                    return True
        elif img.mode == "RGBA":
            extrema = img.getextrema()
            if extrema[3][0] < 255:
                return True

        return False

def create_buttons():
    """creating buttons for image labelling
       used for Image Labelling in Jupyter Notebooks"""
    btns_names = ['Yes', 'No']
    for btn_name in btns_names:
        button_download = widgets.Button(description = btn_name)   
        button_download.on_click(partial(on_clicked, btn_label=btn_name))
        display(button_download)

def preprocess_select_images(dataset, base_images, base_images_dir, target_dir):
    """
    Necessary preprocessing for images retrieved from the internet and 
    finding the most relevant images based on base_images
    
    Args:
    dataset - folder name of the location of images to check
    base_images - list of images to which the images should be similar to
    base_images_dir - the directory at which the base images are located at
    target_dir - the directory where to put selected images
    """
    # Let's define some paths where to save images, tfrecords and embeddings
    path_images = f"{dataset}/images"
    path_tfrecords = f"{dataset}/tfrecords"
    path_embeddings = f"{dataset}/embeddings"
    
    # loading the filenames
    fns_to_check = glob(f'{path_images}/*jpg')
    
    # copying base_images
    owd = os.getcwd()
    for im_fn in base_images:
        copyfile(os.path.join(os.getcwd(), base_images_dir, im_fn), os.path.join(path_images, im_fn))
    os.chdir(owd)
    
    # check if the image is transparent
    for im_fn in glob(f'{path_images}/*jpg'):
        with Image.open(im_fn) as im:
            if has_transparency(im) == True:
    #             display(Image1(im_fn))
                try:
                    os.remove(f_name)
                    print('deleted', f_name)
                except:
                    print('could not delete', f_name)
    
    # loading as PIL objects
    maybe_eyes = []
    for p in fns_to_check:
        try:
            maybe_eyes.append(image.load_img(p))
        except UserWarning:
            print(p)
        
    # resizing to 224x224
    for imgId, img in enumerate(maybe_eyes):
        resized_img = square_resize_image(img, 224)
        resized_img.save(os.path.join(fns_to_check[imgId]))
    
    # removing images that are black and white
    for f_name in glob(f'{path_images}/*jpg'):
        image = imread(f_name)
        if len(image.shape)<3:
              colorType = 'gray'
        elif len(image.shape)==3:
              colorType = 'color'
        else:
              colorType = 'others'
        if colorType == 'gray':
#             display(Image1(f_name))
            os.remove(f_name)
    
    # converting .jpg to .jpeg
    for im_fn in glob(f'{path_images}/*jpg'):
        im = Image.open(im_fn)
        im.save(im_fn[:-4]+'.jpeg')
        os.remove(im_fn)
    
    # convert images to tfrecords
    image_embeddings.inference.write_tfrecord(image_folder=path_images,
                                          output_folder=path_tfrecords,
                                          num_shards=10)
    
    # create embeddings
    image_embeddings.inference.run_inference(tfrecords_folder=path_tfrecords,
                                         output_folder=path_embeddings,
                                         batch_size=1000)
    
    # create vector space with knn
    [id_to_name, name_to_id, embeddings] = image_embeddings.knn.read_embeddings(path_embeddings)
    index = image_embeddingsembeddings.knn.build_index(embeddings)
    
    base_images_ids = tuple(
        name_to_idx[1] for name_to_idx in name_to_id if name_to_idx[0] in base_images)
    
    # getting the knn distance of other images to base images
    [image_embeddings.knn.display_picture(path_images, id_to_name[img_fn]) for img_fn in base_images_ids]
    results = image_embeddings.knn.search(index, id_to_name, sum(embeddings[idx] for idx in base_images_ids), 500)

    def on_clicked(arg, btn_label):
        """the function to execute when a button is clicked
           used for Image Labelling in Jupyter Notebooks"""
        global curImgIdx
        img_fn = results[curImgIdx][1]

        if btn_label == 'Yes':
            copyfile(path_images+'/'+img_fn+'.jpeg', target_dir+img_fn+'.jpeg')

        clear_output(wait=True)


        curImgIdx += 1
        img_fn = results[curImgIdx][1]
        print('current image index: ', curImgIdx)
        img = Image1(filename=path_images+'/'+img_fn+'.jpeg')
        display(img)

        create_buttons()
    
    return on_clicked, results, path_images

In [23]:
on_clicked, results, path_images = preprocess_select_images(
    'maybe_waist_up_portrait_sitting',
    ['0_270.jpg', '2584_0_1.jpg'], 'data/train_csv_224', 'normal_in_camera')

curImgIdx = 0
img_fn = results[curImgIdx][1]
print('current image index: ', curImgIdx)
img = Image1(filename=path_images+'/'+img_fn+'.jpeg')
display(img)

create_buttons()

FileNotFoundError: [Errno 2] No such file or directory: '/data/train_csv_224/0_270.jpg'